In [2]:
DATE = "0620_stronger_aug"
DATA_TO_TEST = 'normal' # 'abn1' or 'abn2' or 'normal'
MODEL_EPOCH = '_25ep' ###### 아래 스냅샷 파일과 일치하는지 반드시 확인!!!!!!! ######

SCORE_VALUE = 0.1
PRINT_BENIGN_DETECTION = False

import os
import pandas as pd

result_save_base_path = '/home/huray/workspace/ncc/results/'
result_save_path = os.path.join(result_save_base_path, DATE, DATA_TO_TEST + MODEL_EPOCH + '_' + str(SCORE_VALUE))
if not os.path.exists(result_save_path):
    os.makedirs(result_save_path)

In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import keras
import keras.preprocessing.image
from keras_retinanet.models.resnet import custom_objects
# from keras_retinanet.preprocessing.coco import CocoGenerator
from keras_retinanet.preprocessing.csv_generator import CSVGenerator

import matplotlib.pyplot as plt
import cv2

import numpy as np
import time

import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
keras.backend.tensorflow_backend.set_session(get_session())

/home/huray/venvs/ml/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load RetinaNet model

In [4]:
model = keras.models.load_model('snapshots/0620_stronger_aug/resnet101_csv_25.h5', custom_objects=custom_objects)

/home/huray/venvs/ml/lib/python3.5/site-packages/keras/models.py:274: UserWarning: Output "nms" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "nms" during training.
  sample_weight_mode=sample_weight_mode)


## Initialize data generators

In [5]:
# create image data generator object
val_image_data_generator = keras.preprocessing.image.ImageDataGenerator()

if DATA_TO_TEST == 'abn1':
    csv_path = '/home/huray/data/NCC/img_retinanet/data_abn1.csv'
elif DATA_TO_TEST == 'abn2':
    csv_path = '/home/huray/data/NCC/img_retinanet/data_abn2.csv'
elif DATA_TO_TEST == 'normal':
    csv_path = '/home/huray/data/NCC/img_retinanet/data_normal.csv'
else:
    print('WRONG DATA CSV PATH!')
    raise

# create a generator for testing data
val_generator = CSVGenerator(
    csv_path,
    '/home/huray/data/NCC/img_retinanet/class_2_onlyM.csv',
    val_image_data_generator,
    batch_size=1
)

test_file_df = pd.read_csv(csv_path, header=None)
num_of_test_files = len(test_file_df)
print(num_of_test_files)

353


## Run detection

In [5]:
image_with_no_detection = []
image_with_more_than_one_anno = []
num_of_anno = 0
correct_detection = 0

In [6]:
img_index_list = list(range(0, num_of_test_files))
img_index_list_temp = img_index_list.copy()
print(len(img_index_list))

353


In [7]:
for index in img_index_list:
    pass_this_file = False
    
    # load image
    image, image_path = val_generator.load_image(index, get_image_path=True)
    print(image_path)

    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = val_generator.preprocess_image(image)
    image, scale = val_generator.resize_image(image)
    annotations = val_generator.load_annotations(index)
    

    # process image
#     start = time.time()
    _, _, detections = model.predict_on_batch(np.expand_dims(image, axis=0))
#     print("processing time: ", time.time() - start)

    # compute predicted labels and scores
    predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
    scores = detections[0, np.arange(detections.shape[1]), 4 + predicted_labels]

    # correct for image scale
    detections[0, :, :4] /= scale
    
    anno_center = []
    
    if len(annotations) > 2:
        print("MANY ANNO!!! len: " +str(len(annotations)) + ' path: '+ image_path )
        image_with_more_than_one_anno.append(image_path)
    
    
    # visualize annotations
    for annotation in annotations:
        num_of_anno += 1
        
        label = int(annotation[4])
        b = annotation[:4].astype(int)
        
        anno_center.append((b[0]+b[2])/2)
        anno_center.append((b[1]+b[3])/2)
        
        
        cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (50, 50, 255), 2)
        caption = "{}".format(val_generator.label_to_name(label))
        cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 5)
        cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 4)
            

    # visualize detections
    number_of_proper_detection = 0
    for idx, (label, score) in enumerate(zip(predicted_labels, scores)):
        if score < SCORE_VALUE:
            continue
            
        number_of_proper_detection += 1

        b = detections[0, idx, :4].astype(int)
            
        caption = "{} {:.3f}".format(val_generator.label_to_name(label), score)
        
        
        ######### BENIGN 출력 여부 설정 #########
        if PRINT_BENIGN_DETECTION is False:
            if 'B' in caption:
                continue
                
                
        if pass_this_file is False and len(anno_center) == 2 and b[0]<anno_center[0]<b[2] and b[1]<anno_center[1]<b[3]:
            correct_detection += 1
            pass_this_file = True
            
            # remove the detected image from the list
            img_index_list_temp.remove(index)
       
        cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 5)
        cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 4)
        cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 5)
            
    if number_of_proper_detection == 0:
        image_with_no_detection.append(image_path)
        
   
    cv2.imwrite(os.path.join(result_save_path, image_path.split('/')[-1].split('.')[0]+'_'+str(index)+'.jpg'), draw)


/home/huray/data/NCC/img_retinanet/normal/normal-00035-30003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00066-30002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00002-30001_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00009-30002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00074-20003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00004-40002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00092-30003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00000-40003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00022-30002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00044-20004_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00016-40002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00052-40001_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00035-30002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00032-40002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00049-30004_0

/home/huray/data/NCC/img_retinanet/normal/normal-00016-40001_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00030-30003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00015-30004_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00079-40004_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00044-20003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00054-30004_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00039-30003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00064-40001_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00037-20004_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00003-40004_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00075-10003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00020-20004_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00047-30001_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00024-20003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00068-40002_0

/home/huray/data/NCC/img_retinanet/normal/normal-00017-40004_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00053-30001_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00051-30003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00003-40002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00054-30001_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00012-30001_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00082-30003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00041-40003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00050-20002_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00066-30004_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00045-30003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00092-30001_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00072-20001_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00076-30003_0.jpg
/home/huray/data/NCC/img_retinanet/normal/normal-00020-20001_0

In [8]:
print("DONE!" + DATA_TO_TEST + MODEL_EPOCH)

DONE!normal_25ep


In [9]:
# check remained images
img_index_list = img_index_list_temp.copy()

print("FAILED TO DETECT:")

detection_failed = []

for img_index in img_index_list:
    image, image_path = val_generator.load_image(img_index, get_image_path=True)
    image = val_generator.preprocess_image(image)
    image, scale = val_generator.resize_image(image)
    annotations = val_generator.load_annotations(img_index)
    
    if len(annotations) != 0: # images that the model failed to detect
        detection_failed.append(image_path)
        
detection_failed.sort()

for path in detection_failed:
    print(path)

FAILED TO DETECT:


In [10]:
print(correct_detection, num_of_anno)
if len(image_with_more_than_one_anno) != 0:
    print("문제있다!!" + str(image_with_more_than_one_anno))

0 0


In [11]:
case_with_no_detection = []

test_file_path_list = test_file_df[0].tolist()

image_with_no_detection.sort()
for l in image_with_no_detection:        
    splitted = l.split('-')[:2]
    case_path = '-'.join(splitted)
    
    if case_path in case_with_no_detection:
        continue
    
    same_case = [k for k in image_with_no_detection if case_path in k]
    if len(same_case) == len([k for k in test_file_path_list if case_path in k]):
        case_with_no_detection.append(case_path)

In [12]:
print(len(case_with_no_detection))

for l in case_with_no_detection:
    print(l)

19
/home/huray/data/NCC/img_retinanet/normal/normal-00003
/home/huray/data/NCC/img_retinanet/normal/normal-00008
/home/huray/data/NCC/img_retinanet/normal/normal-00017
/home/huray/data/NCC/img_retinanet/normal/normal-00019
/home/huray/data/NCC/img_retinanet/normal/normal-00020
/home/huray/data/NCC/img_retinanet/normal/normal-00025
/home/huray/data/NCC/img_retinanet/normal/normal-00034
/home/huray/data/NCC/img_retinanet/normal/normal-00037
/home/huray/data/NCC/img_retinanet/normal/normal-00038
/home/huray/data/NCC/img_retinanet/normal/normal-00039
/home/huray/data/NCC/img_retinanet/normal/normal-00040
/home/huray/data/NCC/img_retinanet/normal/normal-00044
/home/huray/data/NCC/img_retinanet/normal/normal-00046
/home/huray/data/NCC/img_retinanet/normal/normal-00052
/home/huray/data/NCC/img_retinanet/normal/normal-00057
/home/huray/data/NCC/img_retinanet/normal/normal-00073
/home/huray/data/NCC/img_retinanet/normal/normal-00080
/home/huray/data/NCC/img_retinanet/normal/normal-00081
/home/h